In [15]:
import pandas as pd
import numpy as np
df = pd.read_csv('tmdb_data.csv')

FileNotFoundError: [Errno 2] File b'tmdb_data_with_genres.csv' does not exist: b'tmdb_data_with_genres.csv'

In [16]:
keywords = pd.read_csv('keywords.csv')
keywords.head()

,keywords
0,"['saving the world', 'riddle', 'whip', 'treasu..."
1,"['new york', 'rio de janeiro', 'superhero', 'b..."
2,"['tempel', 'shaolin', 'teenager', 'urination',..."
3,"['dc comics', 'crime fighter', 'secret identit..."
4,"['saving the world', 'artificial intelligence'..."


In [14]:
months = {'January': 0,
         'February': 1,
         'March': 2,
         'April': 3,
         'May': 4,
         'June': 5,
         'July': 6,
         'August': 7,
         'September': 8,
         'October': 9,
         'November': 10,
         'December': 11}

In [4]:
df['release_month'] = df['release_date'].apply(lambda x: months[x.split(' ')[0]])

In [5]:
df.head()

,title,year,release_date,cast,crew,rating,overview,budget,total_revenue,tmdb_id,...,History,Horror,Music,Mystery,Romance,Science Fiction,Thriller,War,Western,release_month
0,Harry Potter and the Half-Blood Prince,2009,"July 8, 2009","['Daniel Radcliffe', 'Rupert Grint', 'Emma Wat...","{'Director': ['David Yates'], 'Novel': ['J.K. ...",PG,"As Harry begins his sixth year at Hogwarts, he...",250000000,933959197,767,...,0,0,0,0,0,0,0,0,0,6
1,Terminator Salvation,2009,"May 20, 2009","['Christian Bale', 'Sam Worthington', 'Anton Y...","{'Director': ['McG'], 'Writer': ['Michael Ferr...",PG-13,"All grown up in post-apocalyptic 2018, John Co...",200000000,371353001,534,...,0,0,0,0,0,1,1,0,0,4
2,Indiana Jones and the Kingdom of the Crystal S...,2008,"May 22, 2008","['Harrison Ford', 'Cate Blanchett', 'Shia LaBe...","{'Characters': ['George Lucas', 'Philip Kaufma...",PG-13,"Set during the Cold War, the Soviets – led by ...",185000000,786636033,217,...,0,0,0,0,0,0,0,0,0,4
3,The Incredible Hulk,2008,"June 12, 2008","['Edward Norton', 'Liv Tyler', 'Tim Roth', 'Wi...","{'Characters': ['Stan Lee', 'Jack Kirby'], 'Di...",PG-13,Scientist Bruce Banner scours the planet for a...,150000000,163712074,1724,...,0,0,0,0,0,1,0,0,0,5
4,The Dark Knight,2008,"July 14, 2008","['Christian Bale', 'Michael Caine', 'Heath Led...","{'Director': ['Christopher Nolan'], 'Screenpla...",PG-13,Batman raises the stakes in his war on crime. ...,185000000,1004558444,155,...,0,0,0,0,0,0,1,0,0,6


In [6]:
actors = df['cast'].apply(lambda x: x[1:-1].replace("'", '').split(', '))

In [7]:
import json
# tmp = df['crew'][0]
crew = []
# x = list(json.loads(tmp.replace("'", '"')).values())
# for y in x:
#     crew.extend(y)
# crew
for x in df['crew']: 
    tmp = list(json.loads(x.replace("['", '["').replace("']", '"]').replace("',", '",').replace("{'", '{"').replace("':", '":').replace(" '", ' "')).values())
    tmp_list = []
    for y in tmp:
        tmp_list.extend(y)
    crew.append(list(set(tmp_list)))

In [8]:
ratings = pd.get_dummies(df['rating'])

In [9]:
df = pd.concat([df, ratings], axis=1, sort=False).drop(columns = ['rating'])

In [10]:
studio = pd.get_dummies(df['studio'])

In [11]:
df['total_gross'] = df['total_gross'].apply(lambda x: int(x[1:].replace(',','')))
df['opening_gross'] = df['opening_gross'].apply(lambda x: int(x[1:].replace(',','')))

In [12]:
df.drop(columns=['overview', 'tmdb_id'], inplace=True)

In [13]:
production_companies = df['production_companies'].apply(lambda x: x[1:-1].replace("'", '').replace('"', '').split(', '))

In [14]:
pcd = pd.get_dummies(production_companies.apply(pd.Series).stack()).sum(level=0)
pcd_sum = pcd.sum()

In [15]:
to_drop = []
pcd['other_production_companies'] = pd.DataFrame(np.zeros((900, 1)))
for x in pcd.columns:
    if x != 'other_production_companies' and pcd_sum.loc[x] < 10:
        to_drop.append(x)
        pcd['other_production_companies'] += pcd[x]

In [16]:
pcd.drop(columns=to_drop, inplace=True)

In [17]:
pcd['other_production_companies'] = pcd['other_production_companies'].apply(lambda x: 1 if x != 0 else 0)

In [18]:
df.drop(columns=['opening_date'], inplace=True)

In [19]:
df.drop(columns=['cast', 'crew', 'studio', 'production_companies', 'release_date'], inplace=True)

In [20]:
df.drop(columns=['imdb_id'], inplace=True)

In [21]:
df = pd.concat([df, studio, pcd], axis=1, sort=False)

In [22]:
X = df.drop(columns = ['total_gross', 'total_revenue', 'title', 'opening_gross'])

In [23]:
#y = df.loc[:, ['total_gross', 'total_revenue']]
y = df['total_revenue']

In [24]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [25]:
from sklearn.naive_bayes import GaussianNB
clf = GaussianNB()
clf.fit(X_train, y_train)
GaussianNB(priors=None)
y_pred = clf.predict(X_test)

In [30]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(np.array(y_pred).reshape(-1, 1))
y_pred_new = scaler.transform(np.array(y_pred).reshape(-1, 1))
# scaler.fit(y_test)
y_test_new = scaler.transform(np.array(y_test).reshape(-1, 1)).reshape(-1, 1)

from sklearn.metrics import mean_squared_error
mean_squared_error(y_test_new, y_pred_new)
#mean_squared_error(y_test, y_pred)

0.03974445034383955

In [27]:
list(X.columns)

['year',
 'budget',
 'total_theaters',
 'opening_theaters',
 'runtime',
 'in_collection',
 'Action',
 'Adventure',
 'Animation',
 'Comedy',
 'Crime',
 'Documentary',
 'Drama',
 'Family',
 'Fantasy',
 'History',
 'Horror',
 'Music',
 'Mystery',
 'Romance',
 'Science Fiction',
 'Thriller',
 'War',
 'Western',
 'release_month',
 'G',
 'PG',
 'PG-13',
 'R',
 'A24',
 'Annapurna',
 'BG',
 'BST',
 'BV',
 'CBS',
 'ENTMP',
 'FD',
 'Focus',
 'Fox',
 'FoxS',
 'Free',
 'Gold.',
 'IFC',
 'LG/S',
 'LGF',
 'MGM',
 'Mira.',
 'Neon',
 'ORF',
 'Orion',
 'Over.',
 'P/DW',
 'Par.',
 'ParV',
 'RAtt.',
 'Rela.',
 'Rog.',
 'SGem',
 'SPC',
 'STX',
 'Sony',
 'Studio 8',
 'Sum.',
 'TriS',
 'UA',
 'UAR',
 'Uni.',
 'W/Dim.',
 'WB',
 'WB (NL)',
 'Wein.',
 '20th Century Fox',
 '21 Laps Entertainment',
 'Amblin Entertainment',
 'Apatow Productions',
 'Bad Robot',
 'Big Screen Productions',
 'Blumhouse Productions',
 'Chernin Entertainment',
 'Columbia Pictures',
 'DC Entertainment',
 'Dentsu',
 'Di Bonaventura Pictu